In [1]:
from datetime import datetime
from csv import writer
import requests, bs4
import re

In [2]:
import pandas as pd

In [3]:

res = requests.get('http://nostarch.com')
res.raise_for_status()

In [2]:
nostarchsoup = bs4.BeautifulSoup(res.text)
type(nostarchsoup)

bs4.BeautifulSoup

In [4]:
url = 'https://www.basketball-reference.com/teams/TOR/2019_games.html'

In [5]:
res = requests.get(url, 'html_parser')
res.raise_for_status()

In [6]:
soup = bs4.BeautifulSoup(res.text, 'html.parser')

In [7]:
list(soup.children)

['\n',
 'html',
 '\n',
 <html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>
 <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
 <link href="https://d2p3bygnnzw9w3.cloudfront.net/req/201904031" rel="dns-prefetch"/>
 <!-- no:cookie fast load the css.           -->
 <link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
 <link crossorigin="" href="https://d2cwpp38twqe55.cloudfront.net" rel="preconnect"/>
 <style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetica,arial,sans-serif;color:#000}body{position:relative;background:#c9cbcd;z-index:0;-webkit-text-size-adjust:none;-moz-text-size-adjust:none;-ms-text-size-adjust:none}a img{border:0}ul,li,ol{margin:0;padding:0;list-style-type:none}table th,table td{border:0}iframe{max-width:100%}

In [8]:
reg_season = soup.find('table',{'id':'games'})

In [203]:
datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})

In [81]:
# datesoup = soup.find_all(attrs = {'data-stat':'date_game'})

In [204]:
dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]

In [205]:
gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})

In [206]:
locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']

In [207]:
loc = list(map(lambda x: 1 if x =='' else 0, locs))

In [210]:
len(loc), len(dates)

(82, 82)

In [211]:
gamedates = [datetime.strptime(date, '%a, %b %d, %Y') for date in dates]
    

In [40]:
for gamedate in gamedates:
    url = f'https://www.basketball-reference.com/boxscores/{gamedate.year}{gamedate.month}{gamedate.day}0TOR.html'
    res = requests.get(url)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
    break

In [100]:
gamedates

[datetime.datetime(2018, 10, 17, 0, 0),
 datetime.datetime(2018, 10, 19, 0, 0),
 datetime.datetime(2018, 10, 20, 0, 0),
 datetime.datetime(2018, 10, 22, 0, 0),
 datetime.datetime(2018, 10, 24, 0, 0),
 datetime.datetime(2018, 10, 26, 0, 0),
 datetime.datetime(2018, 10, 29, 0, 0),
 datetime.datetime(2018, 10, 30, 0, 0),
 datetime.datetime(2018, 11, 2, 0, 0),
 datetime.datetime(2018, 11, 4, 0, 0),
 datetime.datetime(2018, 11, 5, 0, 0),
 datetime.datetime(2018, 11, 7, 0, 0),
 datetime.datetime(2018, 11, 10, 0, 0),
 datetime.datetime(2018, 11, 12, 0, 0),
 datetime.datetime(2018, 11, 14, 0, 0),
 datetime.datetime(2018, 11, 16, 0, 0),
 datetime.datetime(2018, 11, 17, 0, 0),
 datetime.datetime(2018, 11, 20, 0, 0),
 datetime.datetime(2018, 11, 21, 0, 0),
 datetime.datetime(2018, 11, 23, 0, 0),
 datetime.datetime(2018, 11, 25, 0, 0),
 datetime.datetime(2018, 11, 27, 0, 0),
 datetime.datetime(2018, 11, 29, 0, 0),
 datetime.datetime(2018, 12, 1, 0, 0),
 datetime.datetime(2018, 12, 3, 0, 0),
 datet

In [102]:
url = 'https://www.basketball-reference.com/boxscores/201810170TOR.html'
res = requests.get(url)
res.raise_for_status()
soup = bs4.BeautifulSoup(res.text, 'html.parser')

In [52]:
efg = soup.find('div',{'id':'all_four_factors'})

In [96]:
efg.find_all('td',{'data-stat':'efg_pct'})[0].get_text()

'.441'

In [104]:
team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
}

In [296]:
def get_4fac(team):
    
    team_abbrevs = {
    'Cleveland Cavaliers': 'CLE',
    'Boston Celtics':'BOS',
    'Washington Wizards':'WAS',
    'Charlotte Hornets':'CHO',
    'Minnesota Timberwolves':'MIN',
    'Dallas Mavericks':'DAL',
    'Milwaukee Bucks':'MIL',
    'Philadelphia 76ers':'PHI',
    'Phoenix Suns':'PHO',
    'Los Angeles Lakers':'LAL',
    'Utah Jazz':'UTA',
    'Sacramento Kings':'SAC',
    'New York Knicks':'NYK',
    'New Orleans Pelicans':'NOP',
    'Detroit Pistons':'DET',
    'Atlanta Hawks':'ATL',
    'Chicago Bulls':'CHI',
    'Miami Heat':'MIA',
    'Memphis Grizzlies':'MEM',
    'Golden State Warriors':'GSW',
    'Denver Nuggets':'DEN',
    'Brooklyn Nets':'BRK',
    'Los Angeles Clippers':'LAC',
    'Portland Trail Blazers':'POR',
    'Indiana Pacers':'IND',
    'San Antonio Spurs':'SAS',
    'Houston Rockets':'HOU',
    'Oklahoma City Thunder':'OKC',
    'Toronto Raptors':'TOR',
    'Orlando Magic':'ORL'
    }
    
    # Formats webscraped dates to match bball reference URLs for later scraping
    def date_formatter(date):

        if len(str(date.month)) != 2:
            month = '0'+ str(date.month)
        else:
            month = str(date.month)
        
        if len(str(date.day)) != 2:
            day = '0' + str(date.day)
        else:
            day = str(date.day)
        
        newdate = str(date.year) + month + day
        return newdate


    dates = []
    efgs = []
    tovpct = []
    orbpct = []
    ftr = []

        
    # Get team page
    url = f'https://www.basketball-reference.com/teams/{team}/2019_games.html'
    res = requests.get(url, 'html_parser')
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
        
    # Get just regular season stats
    reg_season = soup.find('table',{'id':'games'})
        
    # Pull regular season game dates, generate list, turn it into datetime objects
    datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
    gamedates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date' ]
    game_dates = [datetime.strptime(date, '%a, %b %d, %Y') for date in gamedates]
        
        
    # Find regular season game locations, translate to 1 = home, 0 = away
    souplocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
    locs = [entry.get_text() for entry in souplocs if entry.get_text() == '' or entry.get_text() == '@']
    game_locs = list(map(lambda x: 1 if x =='' else 0, locs))
    
    # Find opponent team names
    oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
    opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
    opp_abbrev = [team_abbrevs[opp] for opp in opps]
        
    # Get Results
    soupresults = reg_season.find_all(attrs = {'data-stat':'game_result'})
    results = [entry.get_text() for entry in soupresults if entry.get_text() == 'W' or entry.get_text() == 'L']
    rslts = list(map(lambda x: 1 if x =='W' else 0, results))

    # Get game stats
    for i, gamedate in enumerate(game_dates):
            
        date = date_formatter(gamedate)
        dates.append(date)
            
        if game_locs[i] == 1:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[1].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[1].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[1].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[1].get_text())
        else:
            url = f'https://www.basketball-reference.com/boxscores/{date}0{team_abbrevs[opps[i]]}.html'
            res = requests.get(url)
            res.raise_for_status()
            soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'html.parser')
            fourfac = soup.find('div',{'id':'all_four_factors'})
            efgs.append(fourfac.find_all('td',{'data-stat':'efg_pct'})[0].get_text())
            tovpct.append(fourfac.find_all('td',{'data-stat':'tov_pct'})[0].get_text())
            orbpct.append(fourfac.find_all('td',{'data-stat':'orb_pct'})[0].get_text())
            ftr.append(fourfac.find_all('td',{'data-stat':'ft_rate'})[0].get_text())

        print("Date: " + date + " Opponent: " + team_abbrevs[opps[i]] + " Result: " + str(rslts[i]) + " EFG%: " + efgs[i] + " TOV%: " + tovpct[i] + " ORB%: " + orbpct[i] + " FTR: " + ftr[i])
    
    teamlist= [team] * 82
    data = {'Team': teamlist, "Game Number":list(range(1,83)), "Result":rslts, "Date": dates, "Opponent": opp_abbrev, "EFG%": efgs, "TOV%" : tovpct, "ORB%":orbpct, "FTR":ftr}
    df = pd.DataFrame.from_dict(data)
    df['EFG%'] = df['EFG%'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%'].apply(lambda x: float(x))
    df['ORB%'] = df['ORB%'].apply(lambda x: float(x))
    df['FTR'] = df['FTR'].apply(lambda x: float(x))
    df['TOV%'] = df['TOV%']/100.0
    df['ORB%'] = df['ORB%']/100.0

    
    return df

In [297]:
TOR2019 = get_4fac('TOR')

Date: 20181017 Opponent: CLE Result: 1 EFG%: .565 TOV%: 8.2 ORB%: 18.6 FTR: .130
Date: 20181019 Opponent: BOS Result: 1 EFG%: .538 TOV%: 10.7 ORB%: 26.7 FTR: .152
Date: 20181020 Opponent: WAS Result: 1 EFG%: .500 TOV%: 14.1 ORB%: 33.3 FTR: .330
Date: 20181022 Opponent: CHO Result: 1 EFG%: .661 TOV%: 12.2 ORB%: 15.6 FTR: .138
Date: 20181024 Opponent: MIN Result: 1 EFG%: .579 TOV%: 14.5 ORB%: 28.2 FTR: .101
Date: 20181026 Opponent: DAL Result: 1 EFG%: .544 TOV%: 10.7 ORB%: 26.1 FTR: .187
Date: 20181029 Opponent: MIL Result: 0 EFG%: .422 TOV%: 8.0 ORB%: 23.4 FTR: .214
Date: 20181030 Opponent: PHI Result: 1 EFG%: .571 TOV%: 12.6 ORB%: 23.1 FTR: .261
Date: 20181102 Opponent: PHO Result: 1 EFG%: .580 TOV%: 12.6 ORB%: 14.0 FTR: .160
Date: 20181104 Opponent: LAL Result: 1 EFG%: .551 TOV%: 10.3 ORB%: 28.9 FTR: .133
Date: 20181105 Opponent: UTA Result: 1 EFG%: .645 TOV%: 13.2 ORB%: 20.0 FTR: .151
Date: 20181107 Opponent: SAC Result: 1 EFG%: .522 TOV%: 16.8 ORB%: 34.2 FTR: .236
Date: 20181110 Opp

In [299]:
TOR2019.head()

,Team,Game Number,Result,Date,Opponent,EFG%,TOV%,ORB%,FTR
0,TOR,1,1,20181017,CLE,0.565,0.082,0.186,0.130
1,TOR,2,1,20181019,BOS,0.538,0.107,0.267,0.152
2,TOR,3,1,20181020,WAS,0.500,0.141,0.333,0.330
3,TOR,4,1,20181022,CHO,0.661,0.122,0.156,0.138
4,TOR,5,1,20181024,MIN,0.579,0.145,0.282,0.101


In [298]:
CLE2019 = get_4fac('CLE')

Date: 20181017 Opponent: TOR Result: 0 EFG%: .441 TOV%: 13.5 ORB%: 28.6 FTR: .341
Date: 20181019 Opponent: MIN Result: 0 EFG%: .527 TOV%: 11.7 ORB%: 29.5 FTR: .269
Date: 20181021 Opponent: ATL Result: 0 EFG%: .515 TOV%: 14.6 ORB%: 30.0 FTR: .091
Date: 20181024 Opponent: BRK Result: 0 EFG%: .399 TOV%: 13.5 ORB%: 30.8 FTR: .169
Date: 20181025 Opponent: DET Result: 0 EFG%: .506 TOV%: 8.6 ORB%: 16.7 FTR: .146
Date: 20181027 Opponent: IND Result: 0 EFG%: .506 TOV%: 13.6 ORB%: 44.4 FTR: .233
Date: 20181030 Opponent: ATL Result: 1 EFG%: .572 TOV%: 10.7 ORB%: 28.3 FTR: .367
Date: 20181101 Opponent: DEN Result: 0 EFG%: .440 TOV%: 12.5 ORB%: 30.2 FTR: .217
Date: 20181103 Opponent: CHO Result: 0 EFG%: .494 TOV%: 12.9 ORB%: 7.3 FTR: .173
Date: 20181105 Opponent: ORL Result: 0 EFG%: .575 TOV%: 17.4 ORB%: 21.1 FTR: .100
Date: 20181107 Opponent: OKC Result: 0 EFG%: .421 TOV%: 16.7 ORB%: 24.5 FTR: .124
Date: 20181110 Opponent: CHI Result: 0 EFG%: .473 TOV%: 10.8 ORB%: 31.3 FTR: .132
Date: 20181113 Opp

In [300]:
pd.concat([TOR2019, CLE2019])

,Team,Game Number,Result,Date,Opponent,EFG%,TOV%,ORB%,FTR
0,TOR,1,1,20181017,CLE,0.565,0.082,0.186,0.130
1,TOR,2,1,20181019,BOS,0.538,0.107,0.267,0.152
2,TOR,3,1,20181020,WAS,0.500,0.141,0.333,0.330
3,TOR,4,1,20181022,CHO,0.661,0.122,0.156,0.138
4,TOR,5,1,20181024,MIN,0.579,0.145,0.282,0.101
5,TOR,6,1,20181026,DAL,0.544,0.107,0.261,0.187
6,TOR,7,0,20181029,MIL,0.422,0.080,0.234,0.214
7,TOR,8,1,20181030,PHI,0.571,0.126,0.231,0.261
8,TOR,9,1,20181102,PHO,0.580,0.126,0.140,0.160
9,TOR,10,1,20181104,LAL,0.551,0.103,0.289,0.133


In [305]:
df = pd.DataFrame()

for team in list(team_abbrevs.values()):
    team_db = get_4fac(team)
    df = pd.concat([df, team_db])

Date: 20181017 Opponent: TOR Result: 0 EFG%: .441 TOV%: 13.5 ORB%: 28.6 FTR: .341
Date: 20181019 Opponent: MIN Result: 0 EFG%: .527 TOV%: 11.7 ORB%: 29.5 FTR: .269
Date: 20181021 Opponent: ATL Result: 0 EFG%: .515 TOV%: 14.6 ORB%: 30.0 FTR: .091
Date: 20181024 Opponent: BRK Result: 0 EFG%: .399 TOV%: 13.5 ORB%: 30.8 FTR: .169
Date: 20181025 Opponent: DET Result: 0 EFG%: .506 TOV%: 8.6 ORB%: 16.7 FTR: .146
Date: 20181027 Opponent: IND Result: 0 EFG%: .506 TOV%: 13.6 ORB%: 44.4 FTR: .233
Date: 20181030 Opponent: ATL Result: 1 EFG%: .572 TOV%: 10.7 ORB%: 28.3 FTR: .367
Date: 20181101 Opponent: DEN Result: 0 EFG%: .440 TOV%: 12.5 ORB%: 30.2 FTR: .217
Date: 20181103 Opponent: CHO Result: 0 EFG%: .494 TOV%: 12.9 ORB%: 7.3 FTR: .173
Date: 20181105 Opponent: ORL Result: 0 EFG%: .575 TOV%: 17.4 ORB%: 21.1 FTR: .100
Date: 20181107 Opponent: OKC Result: 0 EFG%: .421 TOV%: 16.7 ORB%: 24.5 FTR: .124
Date: 20181110 Opponent: CHI Result: 0 EFG%: .473 TOV%: 10.8 ORB%: 31.3 FTR: .132
Date: 20181113 Opp

In [306]:
df.to_csv('2019stats.csv', index = False)